<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install openai --quiet

In [ ]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [ ]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [ ]:
def call(system_prompt: str, message: str) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [ ]:
def marketing_campaign_agent(product : str):
  """
     <PRODUCT>{product}</PRODUCT>

     Your task is to create an compelling marketing campaign for the product provided between the <product> tag.
  """
  return call ("your are a marketing campaign export.", marketing_campaign_agent.__doc__.format(product=product) )

In [ ]:
def ready_validation_agent(marketing_text : str) -> bool:
  """
     <CONTENT>{text}</CONTENT>

    I have asked you to create a marketing text for a product earlier. You can find the text between the <CONTENT> tags.

    Is this the final version of the marketing text and ready to publish or can we improve it?

     Your task is to answer with
     - TRUE if this is the final version of the marketing text and ready to publish.
     - FALSE if this is not the final version of the marketing text and we need to improve it.

     Only answer with TRUE or FALSE. add nothing else to your response.

    """
  result = call ("your are a marketing campaign export.", ready_validation_agent.__doc__.format(text=marketing_text) )
  print (result)
  return s.strip().lower() == "true"

In [ ]:
marketing_text = marketing_campaign_agent("iPhone 15")
is_finished=ready_validation_agent(marketing_text)

NameError: name 'FALSE' is not defined

In [ ]:
marketing_text

'Here\'s a comprehensive marketing campaign for the iPhone 15:\n\n**Campaign Name:** "Unlock the Future"\n\n**Objective:** To position the iPhone 15 as the most innovative and desirable smartphone of the year, appealing to tech-savvy customers who crave innovation and excellence.\n\n**Target Audience:**\n\n* Demographics: Tech enthusiasts aged 18-45\n* Psychographics: Individuals who value cutting-edge technology, design, and user experience\n\n**Key Messages:**\n\n1. Experience the future with iPhone 15\'s revolutionary A17 Bionic chip and advanced camera system.\n2. Upgrade your mobile life with 5G connectivity, augmented reality capabilities, and AI-powered features.\n3. Elevate your creativity with a stunning display, impressive battery life, and seamless ecosystem integration.\n\n**Marketing Channels:**\n\n1. **TV Commercial:** "Timeless" - A futuristic-themed ad featuring a time-traveling protagonist discovering the iPhone 15 as the key to unlocking the future.\n2. **Print Advert

In [ ]:
is_finished


'FALSE'